In [ ]:
from mpcontribs.client import Client
from collections import defaultdict

In [ ]:
client = Client()  # initiate with headers/get_consumer on server

In [ ]:
COMPONENTS = ["data", "tables", "structures", "attachments"] # supported contribution components
identifier = "mp-2715"  # "mp-6340"

# initial requests on detail page load

In [ ]:
# get all contribution IDs for this identifier
all_ids = client.get_all_ids({"identifier": identifier})

In [ ]:
# basic project info for all projects
names = list(all_ids.keys())
projects = client.projects.get_entries(
    name__in=names, _fields=["name", "long_title", "authors", "description", "references"]
).result()

In [ ]:
projects#["total_count"]  # total number of projects for this identifier

# click on "Contributions" for a specific project

In [ ]:
name = "carrier_transport" #names[0]  # selected project
ids = list(all_ids[name]["ids"]) # list of contribution ObjectIDs

In [ ]:
# retrieve legend ("other") and list of potential columns
info = client.projects.get_entry(pk=name, _fields=["other", "columns"]).result()
# if set, use info["other"] to show legend explaining columns
info["other"]

In [ ]:
info["columns"]

In [ ]:
data_columns = {} # potential data columns in dot-notation and their units
root_data_columns = defaultdict(set) # potential root-level data columns and their (list of) unit(s)
has_component = {c: False for c in COMPONENTS}  # potentially available components

for column in info["columns"]:
    path, unit = column["path"], column["unit"]
    if path.startswith("data."):
        column = path.split(".", 1)[1]
        clean_unit = None if unit == "NaN" else unit
        data_columns[column] = clean_unit
        root = column.split(".", 1)[0]
        root_data_columns[root].add(clean_unit)
        has_component["data"] = True
    elif path in COMPONENTS[1:]:
        has_component[path] = True

In [ ]:
has_component

## select specific contribution

In [ ]:
cid = ids[0]
# use has_component to show list of potentially available contribution components
has_component

### click on "Data" component

In [ ]:
# use root_data_columns to show a list of root-level columns and their unit(s)
root_data_columns

#### click on root-level data column

In [ ]:
# retrieve full sub-tree of values for the selected root data column
root_column = "PF" # list(root_data_columns.keys())[0]
fields = [
    f"data.{col}" if unit is None else f"data.{col}.display"
    for col, unit in data_columns.items() 
    if col.startswith(root_column)
]
resp = client.contributions.get_entry(pk=cid, _fields=fields).result()

In [ ]:
resp  # use this to display values for requested contribution root data column

### click on "Tables" component

In [ ]:
component = "tables"
resp = client.contributions.get_entry(pk=cid, _fields=[component]).result()

In [ ]:
resp[component] # use this to show list of available tables (and table ObjectIDs)

#### preview specific table

In [ ]:
tid = resp[component][0]["id"]
fields = ["id", "name", "md5", "attrs", "columns", "total_data_rows"]
client.tables.get_entry(pk=tid, _fields=fields).result()

#### download specific table

In [ ]:
# href to download URL
print(f"https://contribs.materialsproject.org/contributions/component/{tid}")

#### (optional/future) visualize table

In [ ]:
table = client.get_table(tid)
table

In [ ]:
table.display()

In [ ]:
resp = client.contributions.get_entry(pk=cid, _fields=["notebook"]).result()
nid = resp["notebook"]["id"]

In [ ]:
nb = client.notebooks.get_entry(pk=nid, _fields=["_all"]).result()

In [ ]:
from base64 import b64decode
from IPython.display import Image

In [ ]:
img = b64decode(nb["cells"][4]["outputs"][0]["data"]["image/png"], validate=True)

In [ ]:
Image(img)